# Bitcoin Mint Verification Notebook
This notebook lets you fetch a Natural Standard image from a short-lived URL, compute its SIFT features, generate the Bitcoin mint address, and display the result.

Run each cell in order. When prompted, paste the short-lived URL that your Flask app generated.

In [ ]:
!pip install opencv-python==4.8.0.76 opencv-contrib-python==4.8.0.76 base58==2.1.1 numpy==1.26.4 requests

In [ ]:
import cv2
import numpy as np
import hashlib
import binascii
import base58
import requests
from IPython.display import display, HTML

In [ ]:
# --- Define helper functions for SIFT and address generation ---
def compute_sift_features_from_array(gray_image):
    # Normalize lighting
    img = cv2.equalizeHist(gray_image)
    sift = cv2.SIFT_create(contrastThreshold=0.04, edgeThreshold=10, nfeatures=200)
    keypoints, descriptors = sift.detectAndCompute(img, None)
    if descriptors is None:
        return np.array([])
    return descriptors.flatten()

def generate_feature_vector_hash(feature_vector):
    counter = 0
    while True:
        rolled = np.roll(feature_vector, counter)
        fv_bytes = rolled.tobytes()
        h = hashlib.sha256(fv_bytes).hexdigest()[:19]
        if '0' not in h and 'I' not in h:
            return h
        counter += 1

def b58ec(hex_str):
    data = bytearray.fromhex(hex_str)
    return base58.b58encode(data).decode('ascii')

def b58dc(encoded, trim=0):
    raw = base58.b58decode(encoded)
    return raw[:-trim] if trim else raw

def hh256(data_bytes):
    first = hashlib.sha256(data_bytes).digest()
    second = hashlib.sha256(first).digest()
    return binascii.hexlify(second)

def burn(template):
    decoded = b58dc(template, trim=4)
    hexed = binascii.hexlify(decoded).decode('ascii')
    check = hh256(decoded)[:8].decode('ascii')
    return b58ec(hexed + check)


## Fetch and Verify

In [ ]:
# Prompt for the short-lived URL of the uploaded Natural Standard image
url = input("Enter the short-lived image URL:").strip()

# Download the bytes
resp = requests.get(url)

# Print some debug info:
print("URL requested:", url)
print("HTTP status code:", resp.status_code)
print("Content-Type header:", resp.headers.get("Content-Type"))  # might be None
print("Response size (bytes):", len(resp.content))

# Build a 1-D uint8 array from the bytes
nparr = np.asarray(bytearray(resp.content), dtype=np.uint8)
print("nparr dtype:", nparr.dtype, "shape:", nparr.shape)

# Decode to grayscale (this will fail if the bytes are not a valid image)
gray = cv2.imdecode(nparr, cv2.IMREAD_GRAYSCALE)
if gray is None:
    raise RuntimeError("cv2.imdecode returned None — make sure the URL points to a real PNG/JPEG")

# Compute SIFT feature vector and hash
fv = compute_sift_features_from_array(gray)
fv_hash = generate_feature_vector_hash(fv)
template = f"1BtcMint{fv_hash}XXXXXXX"
address = burn(template)
print(f"Computed Bitcoin Mint Address: {address}")
